In [ ]:
# This program prints 5 days 1 minute tick data with corresponding volume and TR for each minute and also plots/prints the information about the options related to the stock

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime,timedelta
from matplotlib import dates as mdates
import time
import pytz
import os
from pathlib import Path
import plotly.graph_objects as go

from plotly.subplots import make_subplots


In [ ]:
! pip install -U kaleido #for saving plotly objects

In [ ]:
sns.set_theme(style="darkgrid")

In [ ]:
#local time setting
local_tz = pytz.timezone('US/Eastern')
print(f"current time: {datetime.now(local_tz)}")

In [ ]:
#check if dirs exists otherwise create it
dir_name='stock_information'
directory_path=Path(f"./{dir_name}")
html_path=Path(f"{directory_path}/htmlplot")

if not directory_path.exists():
  directory_path.mkdir(parents=True)
if not html_path.exists():
  html_path.mkdir(parents=True)

opt_path=directory_path


In [ ]:
class minute_data_loader:
    def __init__(self,directory_path,save_dir):
        self.directory_path=directory_path
        #self.save_dir=save_dir

    def download_and_save(self,symbol):
        print(50*"==")
        day=5;interval='1m'
        print(f"downloading and saving {day} days {interval} data for symbol: {symbol}\n")
        data = yf.download(symbol, period=f'{5}d', interval=interval,group_by='ticker')
        dfs={date:group_df for date,group_df in data.groupby(data.index.date)}
        date_values=list(dfs.keys())
        print(f"date values: {date_values}")

        for date in date_values:
            print(f"date: {date} symbol: {symbol}\n")

            temp_df1=dfs[date].round(2)
            temp_df=temp_df1[symbol] #this helps to solve multi index in saving and retriving later
            print(temp_df.head().to_string());print(temp_df.tail().to_string())
            save_file_name=f'{self.directory_path}/{symbol}_{date}.csv'
            print(f"\nsaving data frame as : {save_file_name}\n")
            temp_df.to_csv(f'{save_file_name}')

    def listing_csv_files(self):
        file_list=sorted([f for f in os.listdir(self.directory_path) if f.endswith('.csv')])
        # for filename in file_list:
        #     if filename.endswith(".csv"):
        #         print(filename)
        return file_list

    def list_csv_files_for_symbol(self,symbol):
      '''csv files for given ticker in the given directory'''
      file_list=sorted([f for f in os.listdir(self.directory_path) if f.endswith('.csv') and f.startswith(symbol)])
      return file_list

    # def get_date_time(self,index):
    #     return index.strftime('%H:%M')

    # def filter_time_df(self,df):
    #   df.index=pd.Series(df.index).apply(self.get_date_time)
    #   return df.round(2)

    def get_n_largest_volume_df(self,use_file,resampling=5,count=20,largest=True):
      '''
      get n largest volume of a dataframe in millions with time as index
      '''
      r=f'{resampling}T'

      print(f'reading file: {use_file}')

      df=pd.read_csv(f'{self.directory_path}/{use_file}')

      last_price=df['Adj Close'].iloc[-1];symbol=use_file.split('_')[0]
      print(f'symbol: {symbol}:\nlast_price: {last_price}')

      df['Datetime']=pd.to_datetime(df['Datetime'])
      df=df.set_index('Datetime')


      #print(df.head().to_string())
      #index=df.index
      #symbol=index[0].split('_')[0]
      date=df.index.date[0]
      day_name=df.index.day_name()[0]


      #day_name=df['Datetime'].map(lambda x:x.strftime('%A'))
      print(f"date: {date} {day_name} with {r}")
      temp_df=df.resample(r).sum()
      #total volume
      #plot resampled volume vs time
      # temp_df['Volume'].plot(kind='bar',figsize=(15,8))
      total_volume=round(temp_df.Volume.sum()/10**6,2)
      print(f'Total Volume: {total_volume:.2f}M')

      #max volume
      max_volume_df=temp_df.nlargest(count,'Volume') if largest else temp_df.nsmallest(count,'Volume')
      max_volume_df[f'Volume']=max_volume_df['Volume']/10**6
      max_volume_df[f'{symbol.upper()}_volume_pct_{date}']=max_volume_df['Volume']*100/total_volume
      max_volume_df.rename(columns={'Volume':f'{symbol.upper()}_volume_{date}'},inplace=True)

      max_volume_df.index=max_volume_df.index.map(lambda x:x.strftime('%H:%M'))

      #max_volume_df.index=pd.Series(max_volume_df.index).apply(self.get_date_time)
      max_volume_df=max_volume_df.round(2).reset_index(drop=False)
      #print(max_volume_df.to_string())
      return symbol,last_price,total_volume,day_name,resampling,max_volume_df.iloc[:,[0,6,7]]


    def volume_df_plot(self,use_file,ax,resampling=1,largest=True,count=20):
      r=f'{resampling}T'

      print(f'reading file: {use_file}')

      df=pd.read_csv(f'{self.directory_path}/{use_file}')

      last_price=df['Adj Close'].iloc[-1];symbol=use_file.split('_')[0]
      print(f'symbol: {symbol}:\nlast_price: {last_price}')




      df['Datetime']=pd.to_datetime(df['Datetime'])
      df=df.set_index('Datetime')

      print(df.head().to_string())
      #index=df.index
      #symbol=index[0].split('_')[0]
      date=df.index.date[0]
      day_name=df.index.day_name()[0]


      #day_name=df['Datetime'].map(lambda x:x.strftime('%A'))
      print(f"date: {date} {day_name} with {r}")
      temp_df=df.resample(r).sum()
      #total volume
      #plot resampled volume vs time
      # temp_df['Volume'].plot(kind='bar',figsize=(15,8))
      max_volume=round(temp_df.Volume.sum()/10**6,2)
      print(f'Total Volume: {max_volume:.2f}M')

      #max volume
      max_volume_df=temp_df.nlargest(count,'Volume') if largest else temp_df.nsmallest(count,'Volume')
      max_volume_df['Volume']=max_volume_df['Volume']/10**6
      max_volume_df['volume_pct']=max_volume_df['Volume']*100/max_volume

      max_volume_df.index=max_volume_df.index.map(lambda x:x.strftime('%m-%d-%H:%M'))

      #max_volume_df.index=pd.Series(max_volume_df.index).apply(self.get_date_time)
      max_volume_df=max_volume_df.round(2)

      #print(max_volume_df.to_string())
      ax=max_volume_df.plot(kind='bar',y='Volume',figsize=(12,8))
      high_low='Highest' if largest else 'Lowest'
      ax.set_xlabel('Time');ax.set_ylabel('Volume(M)');ax.set_title(f'Top {count} {high_low} [{resampling}-min] Volume for {symbol} on {date} [{day_name}]')
      #bar labels
      labels=max_volume_df['Volume'].astype(str)+' ['+max_volume_df['volume_pct'].astype(str)+'%]'
      labels=labels.to_list()
      #putting total volume text in the plot
      bbox=dict(facecolor='yellow', edgecolor='red', alpha=0.5,boxstyle='round,pad=0.5')
      ax.text(0.5,0.5,f'Total Volume: {max_volume:0.2f} M [{day_name}]',transform=ax.transAxes,fontsize=15,bbox=bbox)
      #print(labels)
      for container in ax.containers:
        ax.bar_label(container,labels=labels,rotation=60,color='brown',padding=3,fontsize=8)
      plt.tight_layout()
      return ax
      #plt.show()

    def get_6days_volume_plot(self,symbol,resampling=5,count=20,largest=True):

      six_days_list=self.list_csv_files_for_symbol(symbol)[::-1][:5] #It should be 5 for data from yahoo-finance
      print(f'five_days_list: {six_days_list}')

      symbo_list=[]
      last_price_list=[]
      total_volume_list=[]
      max_volume_df_list=[]
      day_name_list=[]
      resample_list=[]

      for i in six_days_list:
        symbol,last_price,total_volume,day_name,resample,df=self.get_n_largest_volume_df(i,resampling=5,count=20,largest=True)
        symbo_list.append(symbol)
        last_price_list.append(last_price)
        total_volume_list.append(total_volume)
        day_name_list.append(day_name)
        resample_list.append(resample)
        max_volume_df_list.append(df)

      #max_volume_df_list
      merged_df=max_volume_df_list[0]
      for i in max_volume_df_list[1:]:
        merged_df=merged_df.merge(i,on='Datetime',how='outer')

      axes=merged_df.set_index('Datetime').plot(kind='bar',figsize=(20,30),subplots=True,layout=(6,2),sharex=False)
      bbox=dict(facecolor='yellow', edgecolor='red', alpha=0.5,boxstyle='round,pad=0.5')
      bbox1=dict(facecolor='pink', edgecolor='red', alpha=0.5,boxstyle='round,pad=0.5')

      for c,ax in enumerate(axes.flatten()):
        ax.set_xlabel('Time');ax.set_ylabel('Volume(M)');
        l=c//2  #label index
        info_text=f'''
        symbol          : {symbo_list[l]}
        last_price      : {last_price_list[l]}
        total_volume : {total_volume_list[l]} M
        day_name      : {day_name_list[l]}
        resample       : {resample_list[l]} min(s)
        '''
        if c%2==0:ax.text(0.5,0.5,info_text,transform=ax.transAxes,fontsize=15,bbox=bbox)
        else:ax.text(0.5,0.5,info_text,transform=ax.transAxes,fontsize=15,bbox=bbox1)
        #bar labels
        for container in ax.containers:
          labels=[f'{bar.get_height()}' if bar.get_height()!=0 else '' for bar in container]
          ax.bar_label(container,rotation=90,color='magenta',padding=3,fontsize=8,labels=labels)
      plt.tight_layout()
      save_file_name=f'{self.directory_path}/{symbol}_6days_volume.pdf'
      print(f'saving plot as: {save_file_name}')
      plt.savefig(save_file_name)
      plt.show()








In [ ]:
# test=minute_data_loader(directory_path,directory_path)
# test.download_and_save('NVDA')

In [ ]:
class option_data_loader:
    def __init__(self,directory_path):
        self.directory_path=directory_path

    def delete_old_files(self,any_dir_path, days_threshold,want_to_remove=False):
    # Calculate the time in seconds for the threshold
      print(f'Deleting files older than {days_threshold} in direcotry {any_dir_path}')
      current_time = time.time()                                # time since Jan 1, 1970 in seconds
      time_threshold = current_time - (days_threshold * 86400)  # 86400 seconds in a day

      #get the list of files in directory
      for filename in os.listdir(any_dir_path):
        file_path=os.path.join(any_dir_path,filename)          #get the file path
        #print(file_path)
        if os.path.isfile(file_path):
          file_mod_time=os.path.getmtime(file_path)              #file modification since Jan 1, 1970 in seconds
          #print(file_mod_time)
          if file_mod_time<time_threshold:                       #checking if the modification time is less than threshold time

            if (want_to_remove):
              print(f'Deleting file: {file_path}')
              os.remove(file_path)

    def percentage_between_two(self,first,second):
      total=first+second
      return round(first/total*100,0),round(second/total*100,0)


      #get option plots
    def get_option_info_for(self,symbol,exp_idx,strike_range=20):
      #exp_idx->0 (first date) etc.

      ticker = yf.Ticker(symbol)  #ticker object

      # Get the expiration dates for options
      expirations = ticker.options
      expiration=expirations[exp_idx]
      print(f"Available expiration dates: {expirations}\n")


      print(f'Applied expiration date: {expiration}')
      # Get the option chain for the selected expiration date
      option_chain = ticker.option_chain(expiration)

      # Separate call and put options data
      calls = option_chain.calls
      puts = option_chain.puts

      #change the time zone for lastTradeDate
      calls['lastTradeDate']=(calls['lastTradeDate']).dt.tz_convert('US/Eastern')
      puts['lastTradeDate']=(puts['lastTradeDate']).dt.tz_convert('US/Eastern')

      #printinfo
      # print(f'call options for {symbol} on {expiration}')
      # print(calls.head().to_string())
      # print(f'put options for {symbol} on {expiration}')
      # print(puts.head().to_string())

      #merging calls and puts dataframe
      merged=pd.merge(calls.copy(),puts.copy(),on='strike',suffixes=('_call','_put'))

      #print(f'merged dataframe:\n{merged.head().to_string()}')

      #select columns
      temp_col=['contractSymbol','volume','openInterest','impliedVolatility','lastTradeDate']
      my_columns=['strike']+[f'{i}_call' for i in temp_col]+[f'{i}_put' for i in temp_col]
      #print(f'my_columns: {my_columns}')

      merged_df=merged[my_columns]
      merged_df.set_index('strike',inplace=True)
      #print(f'merged_df:\n{merged_df.head().to_string()}')
      last_price = ticker.history(period='1d')['Close'].iloc[-1].round(2)

      print(f"The last closing price of {symbol} is: {last_price}")
      #sym=r'$\textcolor{red}{symbol}$'
      template_title=f'{{}} for {symbol} [LAST: {last_price}] for expiration date: {expiration}'

      #print(template_title.format('TEST'))

      #apply the range to strike price

      # if symbol in ['SPY','QQQ','DIA','IWM','NVDA']:strike_range=15
      # if symbol in ['META','TSLA']:strike_range=30
      price_range=f'{last_price-strike_range:.0f}<=strike<={last_price+strike_range:.0f}'
      print(f'price_range: {price_range}')
      merged_df=merged_df.query(price_range)

      return merged_df,template_title,last_price


    def plot_columns_template(self,df,cols,layout,tit,last_price,template_title):

      axes=df[cols].plot(kind='bar',figsize=(20,8),title=template_title.format(tit),subplots=True,sharex=False,layout=layout)


      temp_df=df[cols].reset_index(drop=False) #temporary df
      temp_df_above=temp_df[temp_df['strike']>last_price]
      temp_df_below=temp_df[temp_df['strike']<last_price]

      # print(f'call_oi{temp_df[cols[0]].to_list()}')

      total_sum=temp_df[cols].sum().to_list()
      above_sum=temp_df_above[cols].sum().to_list()
      below_sum=temp_df_below[cols].sum().to_list()


      total_1_pct,total_3_pct=self.percentage_between_two(total_sum[0],total_sum[2])
      total_2_pct,total_4_pct=self.percentage_between_two(total_sum[1],total_sum[3])
      above_1_pct,below_1_pct=self.percentage_between_two(above_sum[0],below_sum[0])
      above_2_pct,below_2_pct=self.percentage_between_two(above_sum[1],below_sum[1])
      above_3_pct,below_3_pct=self.percentage_between_two(above_sum[2],below_sum[2])
      above_4_pct,below_4_pct=self.percentage_between_two(above_sum[3],below_sum[3])

      total_sum_pct=[total_1_pct,total_2_pct,total_3_pct,total_4_pct]
      above_sum_pct=[above_1_pct,above_2_pct,above_3_pct,above_4_pct]
      #above_sum_pct=[above_1_pct,below_1_pct,above_2_pct,below_2_pct]
      below_sum_pct=[below_1_pct,below_2_pct,below_3_pct,below_4_pct]

      #bbox_props = dict(boxstyle="round,pad=0.5", edgecolor="black", facecolor="lightgray")
      for v,ax in enumerate(axes.flatten()):
        # print(f'vline for {last_price}')
        # ax.axvline(x=last_price,color='r',linestyle='--')
        above_below='ABOVE' #if v<2 else 'BELOW'
        total_text=f'TOTAL: {int(total_sum[v]):,} [{total_sum_pct[v]} %]'
        above_text=f'{above_below}: {int(above_sum[v]):,} [{above_sum_pct[v]} %]'
        above_below='BELOW'
        below_text=f'{above_below}: {int(below_sum[v]):,} [{below_sum_pct[v]} %]'
        #print(f'above_below: {above_below}, total_text: {total_text}, above_text: {above_text}, below_text: {below_text}')

        y=-0.4 if v<2 else 1.05
        fontsize=12
        fc='lime' if v<2 else 'lightcoral'
        ax.text(0.0,y,total_text,fontsize=fontsize,color='midnightblue',fontweight='bold',transform=ax.transAxes,bbox=dict(boxstyle='round,pad=0.5',facecolor=fc, alpha=0.25))
        ax.text(0.35,y,above_text,fontsize=fontsize,color='midnightblue',fontweight='bold',transform=ax.transAxes,bbox=dict(boxstyle='round,pad=0.5',facecolor=fc, alpha=0.25))
        ax.text(0.7,y,below_text,fontsize=fontsize,color='midnightblue',fontweight='bold',transform=ax.transAxes,bbox=dict(boxstyle='round,pad=0.5',facecolor=fc, alpha=0.25))
        for container in ax.containers:
          #print(f'container: {container}')
          bar_values = [rect.get_height() for rect in container]
          max_value=max(bar_values)             #max value
          max_index=bar_values.index(max_value) #max index

          ax.bar_label(container, fmt='{:,.0f}', label_type='center',rotation=90,color='k',fontsize=10,fontweight='bold',padding=3.0)
          #Change the color of the bar with the maximum height
          for i, rect in enumerate(container):
              if i == max_index:
                  #rect.set_color('green')
                  rect.set_edgecolor('navy')
                  rect.set_linewidth(2.0)

      plt.figtext(0.99,.1,f'created on: {datetime.now(local_tz)}',transform=ax.transAxes,fontsize=8,rotation=90,fontweight='bold')
      plt.figtext(0.5,0.5,f'{template_title.split(" ")[2]}',fontsize=150,color='gray',alpha=0.1,ha='center',va='center',rotation=30)
      plt.tight_layout()
      return axes
      #plt.show()

    def get_call_put_interest_plot(self,ticker,week_index,run_day,strike_range):
      df_temp,template_title,last_price=self.get_option_info_for(ticker,exp_idx=week_index,strike_range=strike_range)
      #tit='VOLUME AND OPEN INTEREST'
      tit='volume and open intereset'
      ax=self.plot_columns_template(df_temp,['openInterest_call','volume_call','openInterest_put','volume_put'],layout=(2,2),tit=tit.upper(),\
                              last_price=last_price,template_title=template_title)

      #include total value in subplots
      for a in ax.flatten():
        a.set_title('')
        a.legend(loc='upper left')
        #a.axvline(x=0,color='r',linestyle='--')

      # Add custom text to each subplot
      plt.tight_layout()



      #saving into a file
      week_index='-'.join(template_title.rstrip().split('-')[1:])
      save_name=f"{run_day}-{ticker}-exp-{week_index}-voloi"
      print(f'save_name: {opt_path} {save_name}')
      #fig=ax.get_figure()
      df_temp.to_csv(f'{opt_path}{save_name}.csv')

      plt.savefig(f'{opt_path}{save_name}.pdf')
      plt.show()
      #plt.close()
      return ax

    def get_parameter_plot(self,symbol,par,count=10):
      '''
      get n largest volume of a dataframe in millions with time as index
      '''
      df,ind,values=self.get_option_info_for(symbol,0,20)

      temp_df=df.nlargest(count,columns=par).round(2)

      total_val=temp_df[par].sum()
      print(f'total_val: {total_val}')
      print(f'{temp_df.to_string()}')

      ax=temp_df[par].plot(kind='bar',figsize=(15,6),title=f'Top {count} {par} for {symbol} for {ind}')
      ax.set_xticklabels(temp_df.index,rotation=90);ax.set_ylabel(par)
      for container in ax.containers:
        ax.bar_label(container, fmt='{:,.0f}', label_type='edge',rotation=0,color='r',fontsize=8,fontweight='bold')
      ax.set_title(ind.format(f'TOP {count} {par.upper()}'),fontsize=10,fontweight='bold')
      plt.text(0.5,0.5,f'Total top {count} {par}: {total_val:,}',fontsize=18,transform=ax.transAxes,bbox=dict(facecolor='black', alpha=0.75),color='white')
      plt.tight_layout()
      plt.savefig(f'{self.directory_path}/{symbol}_{par}.pdf')
      plt.show()
      return temp_df




In [ ]:
class atr(minute_data_loader,option_data_loader):
  html_dir='htmlplot'
  def __init__(self,directory_path,save_dir):
    super().__init__(directory_path,save_dir)
    self.directory_path=directory_path
    self.save_dir=directory_path



  def get_atr_vwap(self,source_file,resample_time=5):
    print(f'working with get_atr_vwap')
    df_temp=pd.read_csv(f'{self.directory_path}/{source_file}')#,header=[0,1])
    #df_temp.columns=df_temp.columns.droplevel(0) #drop level 0 column and keep level 1 only
    #df_temp=df_temp.reset_index(drop=False)
    #df_temp=df_temp.dropna(subset=['Datetime'])
    print(f'column_names: {df_temp.columns}')
    print(f'column_names: {df_temp.columns.dtype}')
    print(f"source_file: {source_file}")
    ticker=source_file.split('_')[0]
    print(f'ticker: {ticker}')
    df=df_temp.copy()
    print(df.head().to_string())
    df['Volume']=pd.to_numeric(df['Volume'],errors='coerce')

    try:
      df['Volume']=df['Volume']/10**6
    except KeyError as e:
      print(f'KeyError: {e}')
      #df['Volume']=df['Volume']

    #mappling the datetime
    print(f'{df.head().to_string()}')

    df['Datetime']=pd.to_datetime(df['Datetime']).dt.tz_convert('US/Eastern')#.map(lambda x: x.strftime('%x-%H:%M'))

    print(f'{df.head()}')

    #print(f'df:\n{df.head().to_string()}')

    #changing the resampling to 5 mins
    r=f'{resample_time}T'
    print(f'resampling to: {r}')
    df=df.set_index('Datetime')
    df=df.resample(r).agg({'Open':'first','High':'max','Low':'min','Close':'last','Volume':'sum'})
    df=df.reset_index()

    #print(f'======================')
    df['High_low']=df['High']-df['Low']
    df['High_close']=abs(df['High']-df['Close'].shift())
    df['Low_close']=abs(df['Low']-df['Close'].shift())
    df['TR']=df[['High_low','High_close','Low_close']].max(axis=1)
    df['ATR']=df['TR'].rolling(window=5).mean()
    df['ATR']=df['ATR'].round(2)

    #vwap for a day
    df['cumulative_volume']=df['Volume'].cumsum()
    df['cumulative_weighted_price']=df['Close']*df['Volume']
    df['cumulative_price']=df['cumulative_weighted_price'].cumsum()

    df['vwap']=df['cumulative_price']/df['cumulative_volume']

    selected_columns=['Datetime','Open','High','Low','Close','Volume','TR','ATR','vwap']
    df=df[selected_columns]
    df=df.round(2)
    #print(df.head().to_string())
    # df['ATR'].plot(kind='bar',figsize=(20,8))
    # plt.show()
    return df

  def get_atr_for_symbol(self,symbol,resample=5):
    #get list of
    #df=pd.read_csv(f'{self.directory_path}/{self.minute_data}/{symbo}.csv')
    file_list=self.list_csv_files_for_symbol(symbol)[::-1][:5]
    print(f'file_list: {file_list}')
    dfs=[self.get_atr_vwap(i,resample) for i in file_list]
    merged_df=pd.concat(dfs,axis=0) #axis=0 vertical stack, axis=1 horizontal stack
    #merged_df.reset_index(drop=True,inplace=True
    #merged_df.to_csv(f'{self.save_dir}/{symbol}_atr.csv',index=False)
    print(f"merged_df: {merged_df.head()}")
    return merged_df

  def get_atr_plot(self,symbol):
    df=self.get_atr_for_symbol(symbol)
    grouped_df=df.groupby(df['Datetime'].dt.date)

    for c, (key,group) in enumerate(grouped_df):
      #if c!=4:continue
      print(f'key: {key}')
      print(group)
      temp_df=group.copy()
      # hour_group=temp_df.groupby(temp_df['Datetime'].dt.hour)
      # hour_group=temp_df.groupby(pd.Grouper(freq='30T',key='Datetime'))

      day=key.strftime('%A')
      print(f'day: {day}')


      bbox=dict(facecolor='yellow', alpha=0.5, boxstyle='round,pad=0.3')
    #for k,(hour,hour_df) in enumerate(hour_group):
      #if k!=0:continue
      print(40*'-')
      #print(f'hour: {hour}')
      #print(print(f'{hour_df.to_string()}'))
      temp_df.set_index('Datetime',inplace=True)
      temp_df.index=temp_df.index.map(lambda x: x.strftime('%H:%M'))

      columns=['Open','Low','High','Close','TR','ATR','Volume','vwap']
      temp_df=temp_df[columns]

          # Create subplots: 1 row, 3 columns (since there are 3 columns to plot)
      fig, axes = plt.subplots(3,1, figsize=(20, 20))  # (1 row, 3 columns)

      # Plot TR/ATR
      tr_mean=temp_df['TR'].mean()
      temp_df['TR'].plot(ax=axes[0], color='blue', title='TR',kind='bar')
      temp_df['ATR'].plot(ax=axes[0], color='red', title='ATR',kind='line')
      axes[0].axhline(y=tr_mean, color='green', linestyle='--')
      axes[0].text(0.5,0.5,f'mean-ATR: {tr_mean:0.2f}',transform=axes[0].transAxes,bbox=bbox)
      axes[0].set_ylabel('TR/ATR')


      #plot volume
      mean_volume=temp_df['Volume'].mean()
      temp_df['Volume'].plot(ax=axes[1], color='green', title='Volume',kind='bar')
      axes[1].axhline(y=mean_volume, color='red', linestyle='--')
      axes[1].text(0.5,0.5,f'mean-volume: {mean_volume:0.2f} M',transform=axes[1].transAxes,bbox=bbox)
      axes[1].set_ylabel('Volume [M]')

      #plot close
      open_price=temp_df['Open'].iloc[0]
      max_high=temp_df['High'].max()
      min_low=temp_df['Low'].min()
      close=temp_df['Close'].iloc[-1]
      total_volume=temp_df['Volume'].sum()
      #print(f'open: {open_price}')
      # close=hour_df['Close'].iloc[-1]

      close_string=f'''
      open        : {open_price}
      high        : {max_high}
      low         : {min_low}
      Close       : {close}
      total volume: {total_volume} M

      '''
      print(close_string)

      temp_df['Close'].plot(ax=axes[2], color='k', title='Close',kind='line')
      temp_df['vwap'].plot(ax=axes[2], color='y', title='vwap',kind='line')
      axes[2].text(0.5,0.5,close_string,transform=axes[2].transAxes,bbox=bbox)
      axes[2].set_ylabel('Close [$]')
      #axes[2].axhline(y=mean_close, color='g', linestyle='--')

      for ax in axes:
          ax.tick_params(axis='x', rotation=45,labelsize=8)



      plt.suptitle(f'{symbol} [{key}]-[{day}]')

      #save name
      save_name=f'{self.directory_path}/{symbol}_{day}.pdf'
      print(f'saving plot as: {save_name}')
      plt.savefig(f'{save_name}')

      plt.tight_layout()
      plt.show()

  def get_plotly_plot(self,ticker):
    all_files=self.get_atr_for_symbol(ticker,resample=1)
    for c,(date,df) in enumerate(all_files.groupby(all_files['Datetime'].dt.date)):
      #if c!=0:continue
      print(date)
      print(df.head().to_string())

      # Create subplots: row 1 for candlestick, row 2 for volume
      fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                          row_heights=[0.7, 0.15,0.15],  # Adjust the height ratio between the charts
                          vertical_spacing=0.05,   # Space between the charts
                          subplot_titles=("Candlestick Chart", "Volume","True Range"))

      # Candlestick chart in the first row
      candlestick = go.Candlestick(x=df['Datetime'],
                                  open=df['Open'], high=df['High'],
                                  low=df['Low'], close=df['Close'],
                                  name="Candlestick")
      fig.add_trace(candlestick, row=1, col=1)

      # VWAP line in the first row
      vwap_line = go.Scatter(x=df['Datetime'], y=df['vwap'], mode='lines',
                            name='vwap', line=dict(color='orange'))
      fig.add_trace(vwap_line, row=1, col=1)

      # Volume bars in the second row
      volume_bars = go.Bar(x=df['Datetime'], y=df['Volume'],
                          name="Volume", marker_color='blue')
      fig.add_trace(volume_bars, row=2, col=1)

      #ATR in the third row
      atr_bar = go.Bar(x=df['Datetime'], y=df['TR'],
                            name='TR', marker_color='magenta')
      fig.add_trace(atr_bar, row=3, col=1)

      # atr line in the third row
      atr_line = go.Scatter(x=df['Datetime'], y=df['ATR'], mode='lines',
                            name='atr', line=dict(color='green'))
      fig.add_trace(atr_line, row=3, col=1)


      #log for volume and TR
      #fig.update_yaxes(type='log', row=1, col=1)

      fig.update_yaxes(type='log', row=2, col=1)
      fig.update_yaxes(type='log', row=3, col=1)

      # Set layout properties
      fig.update_layout(
          title=f"{ticker} - {date}",
          #xaxis_title="Date",
          yaxis_title="Price",
          yaxis2_title="Volume",
          xaxis_rangeslider_visible=False,  # Hide the range slider
          height=950,  # Total height of the figure
      )

      # Add a fixed position annotation
      open_price=df['Open'].iloc[0]
      max_high=df['High'].max()
      min_low=df['Low'].min()
      close=df['Close'].iloc[-1]
      total_volume=round(df['Volume'].sum(),2)
      #print(f'open: {open_price}')
      # close=hour_df['Close'].iloc[-1]


      close_string=f'''
      <b>open    : {open_price}</b><br>
      <b>high    : {max_high}</b><br>
      <b>low     : {min_low}</b><br>
      <b>Close   : {close}</b><br>
      <b>Volume  : {total_volume} M</b>
      '''
      #print(close_string)

      fig.add_annotation(
          x=0.5,  # Use a relative position (0 to 1 for x-axis)
          y=0.9,  # Use a relative position (0 to 1 for y-axis)
          text=close_string,  # The text to display
          showarrow=False,  # Show arrow pointing to the annotation
          font=dict(size=18, color="black"),
          xref="paper",  # Use paper coordinates for x
          yref="paper",  # Use paper coordinates for y
          # xanchor="center",  # Anchor the x position
          # yanchor="bottom"  # Anchor the y position
      )



      # Show the figure
      save_name=f"{self.directory_path}/{self.html_dir}/{ticker}-{date}.html"

      print(f'saving as: {save_name}')

      fig.write_html(save_name)
      #fig.write_image('NVDA.png')
      fig.show()

In [ ]:
opt_obj=option_data_loader(directory_path)


In [ ]:
# #minute data for the following list
obj=minute_data_loader(directory_path,directory_path)

In [ ]:


#delete the old files if want_to_remove is True
opt_obj.delete_old_files(directory_path,days_threshold=30,want_to_remove=True)

In [ ]:
#atr object

atr_obj=atr(directory_path,directory_path)

In [ ]:
#remove html plot if it is higher than 10 days
atr_obj.delete_old_files(directory_path,10,want_to_remove=True)

# **Enter the ticker of your interest here**

In [ ]:
indices=['SPY','QQQ','DIA','IWM']
sectors=['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']

stocks=['NVDA','TSLA','MSFT','META','AAPL','GOOGL','AMZN','AMD','MSTR']

#following cells is based on the items in ticker: stocks

In [ ]:
downlaod_minute_data=True

if downlaod_minute_data:
  for symbol in stocks:
    obj.download_and_save(symbol)

In [ ]:
for stock in stocks:atr_obj.get_plotly_plot(stock)

In [ ]:
#for stock in stocks:atr_obj.get_atr_plot(stock)

In [ ]:

for i in stocks:
  print(f'i: {i}')
  opt_obj.get_parameter_plot(i,'openInterest_call',count=10)
  opt_obj.get_parameter_plot(i,'openInterest_put',count=10)
  opt_obj.get_parameter_plot(i,'volume_call',count=10)
  opt_obj.get_parameter_plot(i,'volume_put',count=10)

In [ ]:
#This is the date when the programming was run
currnet_datetime=datetime.now(local_tz)
current_month=currnet_datetime.strftime('%b');current_day=currnet_datetime.day
print(f'current_month: {current_month}, current_day: {current_day}')
run_day=f'{current_month.upper()}{current_day}'
print(f'run_day: {run_day}')

In [ ]:

tickers=stocks
for c,ticker in enumerate(tickers):
  #for debugging
  #if ticker not in ['NVDA']:continue
  total_weeks=1
  strike_range=15
  if ticker in indices:
    strike_range=10
    total_weeks=2
  if ticker in ['NVDA']:strike_range=10
  counter=f'{c+1}/{len(tickers)}'


  print(f'{counter:>10} WORKING WITH {ticker:>5} strike_range: {strike_range:<10} total weeks: {total_weeks}\n')
  for i in range(total_weeks):
    opt_obj.get_call_put_interest_plot(ticker,week_index=i,run_day=run_day,strike_range=strike_range)